# Music Generation Using Deep Learning

## Real World Problem

This case-study focuses on generating music automatically using Recurrent Neural Network(RNN).<br> 
We do not necessarily have to be a music expert in order to generate music. Even a non expert can generate a decent quality music using RNN.<br>
We all like to listen interesting music and if there is some way to generate music automatically, particularly decent quality music then it's a big leap in the world of music industry.<br><br>
<b>Task:</b> Our task here is to take some existing music data then train a model using this existing data. The model has to learn the patterns in music that we humans enjoy. Once it learns this, the model should be able to generate new music for us. It cannot simply copy-paste from the training data. It has to understand the patterns of music to generate new music. We here are not expecting our model to generate new music which is of professional quality, but we want it to generate a decent quality music which should be melodious and good to hear.<br><br>
Now, what is music? In short music is nothing but a sequence of musical notes. Our input to the model is a sequence of musical events/notes. Our output will be new sequence of musical events/notes. In this case-study we have limited our self to single instrument music as this is our first cut model. In future, we will extend this to multiple instrument music. 

## Data Source:
1. http://abc.sourceforge.net/NMD/
2. http://trillian.mit.edu/~jc/music/book/oneills/1850/X/

### From first data-source, we have downloaded first two files:
* Jigs (340 tunes)
* Hornpipes (65 tunes)

In [3]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [4]:
data_directory = "../Data/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = '../Data/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [5]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i + 1]
        yield X, Y

In [6]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(128, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [7]:
def training_model(data, epochs = 80):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data/log.csv", index = False)

In [8]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (16, 64, 512)             44544     
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 256)             787456    
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_2 (LSTM)                (16, 64, 128)             197120    
_________________________________________________________________
dropout_2 (Dropout)          (16, 64, 128)             0         
_________________________________________________________________
time_distributed_1 (TimeDist (16, 64, 87)              11223     
___________________________________________________________

/home/ishdutt/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Batch: 1, Loss: 4.46358585357666, Accuracy: 0.0244140625
Batch: 2, Loss: 4.436729907989502, Accuracy: 0.17578125
Batch: 3, Loss: 4.403998374938965, Accuracy: 0.134765625
Batch: 4, Loss: 4.350777626037598, Accuracy: 0.1025390625
Batch: 5, Loss: 4.109460830688477, Accuracy: 0.1435546875
Batch: 6, Loss: 3.847993850708008, Accuracy: 0.166015625
Batch: 7, Loss: 3.7958757877349854, Accuracy: 0.1630859375
Batch: 8, Loss: 3.784571409225464, Accuracy: 0.142578125
Batch: 9, Loss: 3.7929766178131104, Accuracy: 0.13671875
Batch: 10, Loss: 3.6339097023010254, Accuracy: 0.1484375
Batch: 11, Loss: 3.431927442550659, Accuracy: 0.15234375
Batch: 12, Loss: 3.5604872703552246, Accuracy: 0.123046875
Batch: 13, Loss: 3.766331672668457, Accuracy: 0.1162109375
Batch: 14, Loss: 3.5170960426330566, Accuracy: 0.1455078125
Batch: 15, Loss: 3.7265844345092773, Accuracy: 0.1298828125
Batch: 16, Loss: 3.4580650329589844, Accuracy: 0.1572265625
Batch: 17, Loss: 3.3670132160186768, Accuracy: 0.1767578125
Batch: 18, L

Batch: 141, Loss: 2.682910919189453, Accuracy: 0.302734375
Batch: 142, Loss: 2.6132988929748535, Accuracy: 0.3076171875
Batch: 143, Loss: 2.7360193729400635, Accuracy: 0.2841796875
Batch: 144, Loss: 2.7229232788085938, Accuracy: 0.29296875
Batch: 145, Loss: 2.580122709274292, Accuracy: 0.3076171875
Batch: 146, Loss: 2.685756206512451, Accuracy: 0.2802734375
Batch: 147, Loss: 2.6744112968444824, Accuracy: 0.28515625
Batch: 148, Loss: 2.5625901222229004, Accuracy: 0.30078125
Batch: 149, Loss: 2.6719813346862793, Accuracy: 0.28515625
Batch: 150, Loss: 2.6039376258850098, Accuracy: 0.3017578125
Batch: 151, Loss: 2.7212376594543457, Accuracy: 0.28515625
Epoch 2/80
Batch: 1, Loss: 2.451472043991089, Accuracy: 0.32421875
Batch: 2, Loss: 2.351924180984497, Accuracy: 0.337890625
Batch: 3, Loss: 2.5833945274353027, Accuracy: 0.31640625
Batch: 4, Loss: 2.6778080463409424, Accuracy: 0.294921875
Batch: 5, Loss: 2.5320940017700195, Accuracy: 0.33203125
Batch: 6, Loss: 2.3889882564544678, Accuracy: 0

Batch: 130, Loss: 2.180293321609497, Accuracy: 0.427734375
Batch: 131, Loss: 2.0374560356140137, Accuracy: 0.4365234375
Batch: 132, Loss: 2.1280484199523926, Accuracy: 0.4306640625
Batch: 133, Loss: 2.015058755874634, Accuracy: 0.462890625
Batch: 134, Loss: 1.9652308225631714, Accuracy: 0.4599609375
Batch: 135, Loss: 1.9066145420074463, Accuracy: 0.4833984375
Batch: 136, Loss: 1.9156625270843506, Accuracy: 0.4677734375
Batch: 137, Loss: 1.8048200607299805, Accuracy: 0.478515625
Batch: 138, Loss: 1.7581716775894165, Accuracy: 0.5009765625
Batch: 139, Loss: 1.8850350379943848, Accuracy: 0.46484375
Batch: 140, Loss: 1.9436075687408447, Accuracy: 0.4580078125
Batch: 141, Loss: 1.9501351118087769, Accuracy: 0.4736328125
Batch: 142, Loss: 1.914286494255066, Accuracy: 0.4697265625
Batch: 143, Loss: 2.024998426437378, Accuracy: 0.427734375
Batch: 144, Loss: 1.9655182361602783, Accuracy: 0.4609375
Batch: 145, Loss: 1.882387638092041, Accuracy: 0.455078125
Batch: 146, Loss: 1.9959136247634888, A

...

Batch: 35, Loss: 0.4817250967025757, Accuracy: 0.8408203125
Batch: 36, Loss: 0.4880206882953644, Accuracy: 0.85546875
Batch: 37, Loss: 0.48989927768707275, Accuracy: 0.830078125
Batch: 38, Loss: 0.45381999015808105, Accuracy: 0.8515625
Batch: 39, Loss: 0.4841492474079132, Accuracy: 0.8408203125
Batch: 40, Loss: 0.463823139667511, Accuracy: 0.8408203125
Batch: 41, Loss: 0.42976370453834534, Accuracy: 0.8544921875
Batch: 42, Loss: 0.3606160879135132, Accuracy: 0.8779296875
Batch: 43, Loss: 0.4361276626586914, Accuracy: 0.84375
Batch: 44, Loss: 0.42778992652893066, Accuracy: 0.8564453125
Batch: 45, Loss: 0.414026141166687, Accuracy: 0.8623046875
Batch: 46, Loss: 0.40365105867385864, Accuracy: 0.861328125
Batch: 47, Loss: 0.3979443609714508, Accuracy: 0.8662109375
Batch: 48, Loss: 0.43126678466796875, Accuracy: 0.87109375
Batch: 49, Loss: 0.4437916874885559, Accuracy: 0.8583984375
Batch: 50, Loss: 0.478709876537323, Accuracy: 0.8369140625
Batch: 51, Loss: 0.47224920988082886, Accuracy: 0.8

Batch: 22, Loss: 0.5113568305969238, Accuracy: 0.8330078125
Batch: 23, Loss: 0.4891752600669861, Accuracy: 0.8271484375
Batch: 24, Loss: 0.5410126447677612, Accuracy: 0.818359375
Batch: 25, Loss: 0.48464420437812805, Accuracy: 0.83984375
Batch: 26, Loss: 0.4128234386444092, Accuracy: 0.857421875
Batch: 27, Loss: 0.41259825229644775, Accuracy: 0.8583984375
Batch: 28, Loss: 0.483277827501297, Accuracy: 0.841796875
Batch: 29, Loss: 0.4740712642669678, Accuracy: 0.8388671875
Batch: 30, Loss: 0.40598517656326294, Accuracy: 0.8671875
Batch: 31, Loss: 0.43231505155563354, Accuracy: 0.865234375
Batch: 32, Loss: 0.424957275390625, Accuracy: 0.8486328125
Batch: 33, Loss: 0.500510036945343, Accuracy: 0.84375
Batch: 34, Loss: 0.5316116809844971, Accuracy: 0.8212890625
Batch: 35, Loss: 0.46905505657196045, Accuracy: 0.85546875
Batch: 36, Loss: 0.47865068912506104, Accuracy: 0.8525390625
Batch: 37, Loss: 0.4827432632446289, Accuracy: 0.84375
Batch: 38, Loss: 0.48316964507102966, Accuracy: 0.83886718

Batch: 9, Loss: 0.4874427020549774, Accuracy: 0.83203125
Batch: 10, Loss: 0.4480842649936676, Accuracy: 0.833984375
Batch: 11, Loss: 0.5072295665740967, Accuracy: 0.82421875
Batch: 12, Loss: 0.4572807550430298, Accuracy: 0.84375
Batch: 13, Loss: 0.4182606339454651, Accuracy: 0.859375
Batch: 14, Loss: 0.48700156807899475, Accuracy: 0.8369140625
Batch: 15, Loss: 0.45442283153533936, Accuracy: 0.8623046875
Batch: 16, Loss: 0.4282756447792053, Accuracy: 0.8603515625
Batch: 17, Loss: 0.4730927348136902, Accuracy: 0.828125
Batch: 18, Loss: 0.46243053674697876, Accuracy: 0.8427734375
Batch: 19, Loss: 0.505537748336792, Accuracy: 0.83984375
Batch: 20, Loss: 0.40910837054252625, Accuracy: 0.876953125
Batch: 21, Loss: 0.47452759742736816, Accuracy: 0.837890625
Batch: 22, Loss: 0.5253796577453613, Accuracy: 0.8310546875
Batch: 23, Loss: 0.5274539589881897, Accuracy: 0.8212890625
Batch: 24, Loss: 0.5043020248413086, Accuracy: 0.8349609375
Batch: 25, Loss: 0.4591420888900757, Accuracy: 0.84765625
B

Batch: 147, Loss: 0.4395917057991028, Accuracy: 0.8525390625
Batch: 148, Loss: 0.5370088815689087, Accuracy: 0.8349609375
Batch: 149, Loss: 0.4612531363964081, Accuracy: 0.8427734375
Batch: 150, Loss: 0.49950775504112244, Accuracy: 0.8388671875
Batch: 151, Loss: 0.4476983845233917, Accuracy: 0.853515625
Epoch 80/80
Batch: 1, Loss: 0.6252017021179199, Accuracy: 0.806640625
Batch: 2, Loss: 0.5453973412513733, Accuracy: 0.8095703125
Batch: 3, Loss: 0.5045788884162903, Accuracy: 0.8330078125
Batch: 4, Loss: 0.44380176067352295, Accuracy: 0.857421875
Batch: 5, Loss: 0.48037242889404297, Accuracy: 0.8427734375
Batch: 6, Loss: 0.4947843849658966, Accuracy: 0.830078125
Batch: 7, Loss: 0.497312068939209, Accuracy: 0.8291015625
Batch: 8, Loss: 0.4479140043258667, Accuracy: 0.8427734375
Batch: 9, Loss: 0.4572582244873047, Accuracy: 0.8515625
Batch: 10, Loss: 0.45365074276924133, Accuracy: 0.8408203125
Batch: 11, Loss: 0.515255331993103, Accuracy: 0.8203125
Batch: 12, Loss: 0.44455355405807495, Ac

Batch: 134, Loss: 0.4702020287513733, Accuracy: 0.833984375
Batch: 135, Loss: 0.4212133288383484, Accuracy: 0.861328125
Batch: 136, Loss: 0.47045063972473145, Accuracy: 0.8349609375
Batch: 137, Loss: 0.48452287912368774, Accuracy: 0.8212890625
Batch: 138, Loss: 0.43101686239242554, Accuracy: 0.85546875
Batch: 139, Loss: 0.4941263198852539, Accuracy: 0.83984375
Batch: 140, Loss: 0.48511528968811035, Accuracy: 0.8369140625
Batch: 141, Loss: 0.5163055658340454, Accuracy: 0.8271484375
Batch: 142, Loss: 0.49444669485092163, Accuracy: 0.830078125
Batch: 143, Loss: 0.44614630937576294, Accuracy: 0.857421875
Batch: 144, Loss: 0.4926169216632843, Accuracy: 0.8369140625
Batch: 145, Loss: 0.43183720111846924, Accuracy: 0.8525390625
Batch: 146, Loss: 0.47125592827796936, Accuracy: 0.837890625
Batch: 147, Loss: 0.4637753963470459, Accuracy: 0.8525390625
Batch: 148, Loss: 0.5029866695404053, Accuracy: 0.84375
Batch: 149, Loss: 0.4478471875190735, Accuracy: 0.859375
Batch: 150, Loss: 0.45663452148437

In [9]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,2.721238,0.285156
1,2,1.932443,0.476562
2,3,1.624641,0.543945
3,4,1.458915,0.559570
4,5,1.369484,0.580078
5,6,1.289247,0.607422
6,7,1.227129,0.625977
7,8,1.177866,0.633789
8,9,1.123210,0.653320
9,10,1.084698,0.663086
